In [23]:
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('notebook')
import h5py
import allel; print('scikit-allel', allel.__version__)

scikit-allel 1.1.9


In [3]:
from os.path import join 
WORKDIR = '/home/sergio/Res_CIML/TLX3_project'
SCRIPTS = join(WORKDIR,'scripts')
DATADIR = join(WORKDIR,'data')

In [ ]:
### Functions
def chrom2num(st):
    chrm = st.split(':')[0]
    pos = st.split(':')[1].split('-')

    pl = int(pos[0].replace(',',''))

    pr = int(pos[1].replace(',',''))
    
    return chrm, pl, pr

def plot_variant_density(pos, window_size, title=None):
    
    # setup windows 
    bins = np.arange(pos.min(), pos.max(), window_size)
    
    # use window midpoints as x coordinate
    x = (bins[1:] + bins[:-1])/2
    
    # compute variant density in each window
    h, _ = np.histogram(pos, bins=bins)
    y = h / window_size
    
    # plot
    fig, ax = plt.subplots(figsize=(12, 3))
    sns.despine(ax=ax, offset=10)
    ax.plot(x, y)
    ax.set_xlabel('Chromosome position (bp)')
    ax.set_ylabel('Variant density (bp$^{-1}$)')
    if title:
        ax.set_title(title)

def plot_variant_hist_2d(f1, f2, variants, downsample):
    x = variants[f1][:][::downsample]
    y = variants[f2][:][::downsample]
    fig, ax = plt.subplots(figsize=(6, 6))
    sns.despine(ax=ax, offset=10)
    ax.hexbin(x, y, gridsize=20)
    ax.set_xlabel(f1)
    ax.set_ylabel(f2)
    ax.set_title('Variant %s versus %s joint distribution' % (f1, f2))

def plot_variant_hist(f, variants, bins=30, down=200):
    x = variants[f][:][::down]
    fig, ax = plt.subplots(figsize=(7, 5))
    sns.despine(ax=ax, offset=10)
    ax.hist(x, bins=bins)
    ax.set_xlabel(f)
    ax.set_ylabel('No. variants')
    ax.set_title('Variant %s distribution' % f)

In [ ]:
### VCF direct
fwes =  join(DATADIR,'tracks/WGS-WES/Exomes/WES_TLX3_TAP_Tum7-8.vcf.gz')
fwesh5 = join(DATADIR, 'tracks/WGS-WES/Exomes/WES_TLX3_TAP_Tum7-8.h5')

# read VCF file, transform SNPEFF to separated fields (optional)
cs = allel.read_vcf(fwes,fields='*', numbers={'ALT': 4},transformers=allel.ANNTransformer())


In [ ]:
# cs['samples']
# array(['TAP-01', 'TLX3-01', 'Tumor7-01', 'Tumor8-01'], dtype=object)

In [ ]:
# variants data to DataFrame, transform SNPEFF to separated fields (optional)
var = allel.vcf_to_dataframe(fwes,fields='*', numbers={'ALT': 4}, transformers=allel.ANNTransformer())

In [ ]:
## SNP calculation
#var['REF_len'] = var.apply(lambda row: len(row['REF']), axis=1)
#var['ALT1_len'] = var.apply(lambda row: len(row['ALT_1']), axis=1)
#var['ALT2_len'] = var.apply(lambda row: len(row['ALT_2']), axis=1)

var_snp = var[var['is_snp']==True]



In [ ]:
len(var_snp)

In [ ]:
# Genotype array to special class GenotypeArray
gt = allel.GenotypeArray(cs['calldata/GT'])
    ##- typical functions
    # gt.is_het()
    # gt.count_het(axis=1)
    # ac = gt.count_alleles()

In [ ]:
#var.head()


#gt1 = gt.subset([2,7,9,12,45,67,124])
#gt2= gt.subset([3,8,10,13,46,68,125])

#gt3 = gt2.concatenate(gt1)



#gt.is_het()

In [ ]:
#print(sorted(var['ANN_Feature_Type'].unique()))
#print(sorted(var['ANN_Transcript_BioType'].unique()))

In [ ]:
#cod_var = var[var['ANN_Feature_Type']=='transcript']

In [ ]:
#print(len(var))
#len(cod_var)
#cod_var[['ANN_Gene_Name','ANN_Feature_Type', 'ANN_Transcript_BioType']].head()

In [ ]:
#cod_ind=cod_var.index
#cod_gt=gt[cod_ind]

## Now we have pair {var, gt} for WES 

In [ ]:
#cod_var.head(12)
a,b,c = plt.hist(np.log(var['QUAL']), bins=100)

In [ ]:
#cod_gt[2,:]

## Strip var data to region 

In [ ]:
#st  ='chr12:77,033,211-78,041,433'
#c,l,r = chrom2num(st)
#print(c,l,r)

In [ ]:
#var_reg = var[(var['CHROM']==c) & (var['POS']>l) & (var['POS']<r)]

In [ ]:
# plot_variant_density(var_reg['POS'], window_size=35, title=c)

In [ ]:
### Plot density for all chromosomes
# for ch in df['CHROM'].unique():
#     dfc = df[df['CHROM']==ch]
#    plot_windowed_variant_density(dfc['POS'], window_size=100000, title=ch+' , raw variant density')

## Working with chunked table, we need HDF5 file

In [ ]:
### Save to hdf5
# fwesh5 ='tracks/WGS-WES/Exomes/WES_TLX3_TAP_Tum7-8.h5'

import sys

allel.vcf_to_hdf5(fwes,fwesh5, 
                   fields='*', alt_number=4,transformers=allel.ANNTransformer(),log=sys.stdout, vlen=False)


#csh = h5py.File(fwesh5,mode='r')

In [ ]:
#list(csh['variants'])

In [ ]:
### HDF5 from VCF database

# read HDF5 file
csh = h5py.File(fwesh5,mode='r')
var_tb = allel.VariantChunkedTable(csh['variants'], 
                                   names=['CHROM', 'POS', 'REF', 'ALT', 'DP', 'MQ', 'QD', 'is_snp',
                                             'ANN_AA_length',
                                             'ANN_Allele',
                                             'ANN_Annotation',
                                             'ANN_Annotation_Impact',
                                             'ANN_Feature_ID',
                                             'ANN_Feature_Type',
                                             'ANN_Gene_ID',
                                             'ANN_Gene_Name',
                                             'ANN_Rank',
                                             'ANN_Transcript_BioType','numalt'])

In [ ]:
#var_tb
#a,b,c=plt.hist(var_tb['DP'][:], bins=10)
#csh['variants/REF']

## Now we can work with filters

In [ ]:
#fltr_expr = '(QD > 5) & (MQ > 40) & (DP > 1500) & (DP < 3000)'
#fltr_expr = '(QD > 5) & (MQ > 40) & (DP > 1500) & (DP < 3000)'
fltr_expr="is_snp==True"

var_tb_fltr = var_tb.eval(fltr_expr)[:]

#var_tb
#var_tb_fltr
#np.count_nonzero(var_tb_fltr)
np.count_nonzero(~var_tb_fltr)

#list(csh['calldata'].keys())
#list(csh['variants'].keys())

In [ ]:
var_tb['ANN_Annotation_Impact'][:]

In [ ]:
## apply filter
var_snp = var_tb.compress(var_tb_fltr)


## Genotype from HDF5 

In [ ]:
list(csh['calldata'].keys())

In [ ]:
gth = allel.GenotypeChunkedArray(csh['calldata/GT'])
#gth

In [ ]:
list(csh['samples'])

In [ ]:
import pandas as pd
samples = pd.DataFrame({'sample':list(csh['samples']), 'cell_type':['TAP','TLX3', 'Tumor7','Tumor8']})
TLX = samples['cell_type'].isin(['TLX3'])
TAP = samples['cell_type'].isin(['TAP'])
Tum7 = samples['cell_type'].isin(['Tumor7'])
Tum8 = samples['cell_type'].isin(['Tumor8'])


#TLX

## Subset genotype  samples

In [ ]:
gth_tlx = gth.subset(None,TLX)
gth_tap = gth.subset(None,TAP)
gth_tum7 = gth.subset(None,Tum7)
gth_tum8 = gth.subset(None,Tum8)

In [ ]:
gth_tlx_snp = gth.subset(var_tb_fltr,TLX)
gth_tap_snp = gth.subset(var_tb_fltr,TAP)
gth_tum7_snp = gth.subset(var_tb_fltr,Tum7)
gth_tum8_snp = gth.subset(var_tb_fltr,Tum8)

#### Now we have  tables for WES 

In [ ]:
n_variants = len(var_tb)
pc_missing_tlx = gth_tlx.count_missing(axis=0)[:] * 100 / n_variants
pc_het_tlx = gth_tlx.count_het(axis=0)[:] * 100 / n_variants

pc_missing_tap = gth_tap.count_missing(axis=0)[:] * 100 / n_variants
pc_het_tap = gth_tap.count_het(axis=0)[:] * 100 / n_variants


pc_missing_tum7 = gth_tum7.count_missing(axis=0)[:] * 100 / n_variants
pc_het_tum7 = gth_tum7.count_het(axis=0)[:] * 100 / n_variants

pc_missing_tum8 = gth_tum8.count_missing(axis=0)[:] * 100 / n_variants
pc_het_tum8 = gth_tum8.count_het(axis=0)[:] * 100 / n_variants

print('Total namber of var = ', n_variants)
print('============================')

print('TLX3 missing = ', pc_missing_tlx, '%')
print('TLX3 hetero = ', pc_het_tlx, '%')

print('============================')
print('TAP missing = ', pc_missing_tap, '%')
print('TAP hetero = ', pc_het_tap, '%')

print('============================')
print('Tumor7 missing = ', pc_missing_tum7, '%')
print('Tumor7 hetero = ', pc_het_tum7, '%')

print('============================')
print('Tumor8 missing = ', pc_missing_tum8, '%')
print('Tumor8 hetero = ', pc_het_tum8, '%')


In [ ]:
n_snp = len(var_snp)
tlx_snp_miss = gth_tlx_snp.count_missing(axis=0)[:] * 100 / n_snp
tlx_snp_het = gth_tlx_snp.count_het(axis=0)[:] * 100 / n_snp

tap_snp_miss = gth_tap_snp.count_missing(axis=0)[:] * 100 / n_snp
tap_snp_het = gth_tap_snp.count_het(axis=0)[:] * 100 / n_snp

tum7_snp_miss = gth_tum7_snp.count_missing(axis=0)[:] * 100 / n_snp
tum7_snp_het = gth_tum7_snp.count_het(axis=0)[:] * 100 / n_snp

tum8_snp_miss = gth_tum8_snp.count_missing(axis=0)[:] * 100 / n_snp
tum8_snp_het = gth_tum8_snp.count_het(axis=0)[:] * 100 / n_snp


print('Total namber of SNPs = ', n_snp)
print('============================')
print('TLX3 SNPs missing = ', tlx_snp_miss, '%')
print('TLX3 SNPs hetero = ', tlx_snp_het, '%')

print('============================')
print('TAP SNPs missing = ', tap_snp_miss, '%')
print('TAP SNPs hetero = ', tap_snp_het, '%')

print('============================')
print('Tumor7 SNPs missing = ', tum7_snp_miss, '%')
print('Tumor7 SNPs hetero = ', tum7_snp_het, '%')

print('============================')
print('Tumor8 SNPs missing = ', tum8_snp_miss, '%')
print('Tumor8 SNPs hetero = ', tum8_snp_het, '%')

In [ ]:
tlx_seg = gth_tlx.count_alleles().count_segregating()
tap_seg = gth_tap.count_alleles().count_segregating()
all_seg = gth.count_alleles().count_segregating()

print('ALL segregating = ', all_seg)
print('TLX segregating = ', tlx_seg)
print('TAP segregating = ', tap_seg)
print('num variants = ', n_variants)

In [ ]:
#gth.count_alleles().is_segregating()[2000:2000+6]
#gth[2000:2000+6]

#var['REF']

# Region based files

In [20]:
import pybedtools as pb

WES = 'tracks/WGS-WES/Exomes'
WGS = 'tracks/WGS-WES/Germline'

# Exome vcf
tlx_ex = pb.BedTool(join(DATADIR,WES,'WES_TLX3.vcf.gz'))
tap_ex = pb.BedTool(join(DATADIR,WES,'WES_TAP.vcf.gz'))
tum7_ex = pb.BedTool(join(DATADIR,WES,'WES_Tumor7.vcf.gz'))
tum8_ex = pb.BedTool(join(DATADIR,WES,'WES_Tumor8.vcf.gz'))

In [21]:
# Genome vcf
tlx_gn = pb.BedTool(join(DATADIR,WGS,'TLX3_WGS.vcf.gz'))
tap_gn = pb.BedTool(join(DATADIR,WGS,'TAP_WGS.vcf.gz'))

In [14]:
print('Exome TLX3 = ', len(tlx_ex))
print('Genome TLX3 = ', len(tlx_gn))
print('--------------------------')
print('Exome TAP = ', len(tap_ex))
print('Genome TAP = ', len(tap_gn))
print('========================')

print('Exome Tumor7 = ', len(tum7_ex))
print('Exome Tumor8 = ', len(tum8_ex))

Exome TLX3 =  16415
Genome TLX3 =  1073273
--------------------------
Exome TAP =  9970
Genome TAP =  397878
Exome Tumor7 =  34765
Exome Tumor8 =  87267


# Intersections

In [15]:
tlx_ex_gn = tlx_gn.intersect(tlx_ex, header=True)

In [16]:
tap_ex_gn = tap_gn.intersect(tap_ex, header=True)

In [17]:
print('Common exome an genome TLX3 = ', len(tlx_ex_gn))
print('Common exome an genome TAP = ', len(tap_ex_gn))

Common exome an genome TLX3 =  4244
Common exome an genome TAP =  2521


In [6]:
isec_txp = tap_ex.intersect(tlx_ex, header=True)#.sort()

In [7]:
isec_tum = tum8_ex.intersect(tum7_ex, header=True) #.sort()

In [8]:
print('TLX3 and TAP common = ', len(isec_txp))
print('Tumor common = ', len(isec_tum))

TLX3 and TAP common =  3404
Tumor common =  27714


In [9]:
isec_ex = isec_tum.intersect(isec_txp, header=True)

In [10]:
len(isec_ex)
isec_ex.saveas(join(DATADIR,WES,'Vars_isec_all.vcf'))

<BedTool(/home/sergio/Res_CIML/TLX3_project/data/tracks/WGS-WES/Exomes/Vars_isec_all.vcf)>

# Variant effect by genes

In [24]:
gen_var =  pd.read_table(join(DATADIR,WES,'snpeff/all.genes.txt'), skiprows=1)
gen_var.rename(columns={'#GeneName':'GeneName'}, inplace=True)

In [26]:
#list(gen_var.columns)

In [29]:
gen_var_impact = gen_var.sort_values('variants_impact_HIGH', axis=0, ascending=False)
gen_var_impact.head(10)

,GeneName,GeneId,TranscriptId,BioType,variants_impact_HIGH,variants_impact_LOW,variants_impact_MODERATE,variants_impact_MODIFIER,variants_effect_3_prime_UTR_variant,variants_effect_5_prime_UTR_premature_start_codon_gain_variant,...,variants_effect_non_coding_transcript_variant,variants_effect_splice_acceptor_variant,variants_effect_splice_donor_variant,variants_effect_splice_region_variant,variants_effect_start_lost,variants_effect_stop_gained,variants_effect_stop_lost,variants_effect_stop_retained_variant,variants_effect_synonymous_variant,variants_effect_upstream_gene_variant
20182,Ttn,Ttn,NM_011652.3,protein_coding,13,5,17,15,0,0,...,0,0,0,0,0,0,0,0,5,0
20183,Ttn,Ttn,NM_028004.2,protein_coding,9,5,13,23,0,0,...,0,0,0,0,0,0,0,0,5,0
12939,Obscn,Obscn,NM_199152.3,protein_coding,9,100,46,110,0,0,...,0,0,0,12,0,0,0,0,90,0
11844,Mtus1,Mtus1,NM_001005863.2,protein_coding,8,11,20,26,0,0,...,0,0,1,0,0,0,0,0,11,0
11847,Mtus1,Mtus1,NM_001286413.1,protein_coding,8,11,20,26,0,0,...,0,0,1,0,0,0,0,0,11,0
2635,Brca2,Brca2,NM_001081001.2,protein_coding,7,1,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2636,Brca2,Brca2,NM_009765.3,protein_coding,7,1,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11629,Mroh2a,Mroh2a,NM_001281466.1,protein_coding,7,67,57,289,4,0,...,0,0,0,12,0,6,0,0,60,18
12188,Nckap1l,Nckap1l,NM_153505.4,protein_coding,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19914,Trio,Trio,NM_001081302.1,protein_coding,6,1,0,2,0,0,...,0,0,2,1,0,0,0,0,0,0


# BRCA Function
Mutations in the *BRCA-1* and *BRCA-2* genes are associated with a subset of breast and ovarian cancers. These two genes have different functions within cells. Like the other tumor suppressors discussed so far, mutations can arise spontaneously or they may be inherited. Individuals who inherit a BRCA-1 or BRCA-2 mutation are known to be more susceptible to developing breast cancer. Individuals carrying a BRCA mutation have a lifetime risk (if they live to the age of 85) of 80% for developing breast cancer. The lifetime risks for developing ovarian cancer is 10-20% for BRCA-2 mutations and 40-60% for BRCA-1 mutations. The presence of these mutations may also increase the risk of prostate, pancreatic, colon, and other cancers.The total risk for any person depends on the individual genetic and environmental risk factors to which they are exposed. BRCA-1 and BRCA-2 mutations are thought to be associated with 5-10% of all breast cancers.

In [ ]:
list(gen_var_impact.head(50)['GeneName'].unique())

# CASE 1: Variants for list of genes - tumor suppressors

In [ ]:
import EnrichRLib as erl

tall_sup = erl.read_gmt('gene_lists/T-ALL_suppressor.gmt')
gl = tall_sup['T-ALL-suppressor']

gl

In [ ]:
# working with pair {cod_var, cod_gt}
cod_var.loc[:,'ANN_Gene_Name'] = cod_var['ANN_Gene_Name'].str.upper()

In [ ]:
#cod_var['ANN_Gene_Name'].head()

cod_var_gs = cod_var.loc[cod_var['ANN_Gene_Name'].isin(gl)]

In [ ]:
cod_gt_gs = cod_gt[cod_var_gs.index]


In [ ]:
# TLX3 count homo/hetero
tlx_homalt = cod_gt_gs[:,2:].is_hom_alt()[:]
cod_gt_gs[:,2:].count_hom_alt()


In [ ]:
# TAP count homo/hetero
tap_homalt = cod_gt_gs[:,:2].is_hom_alt()[:]

cod_gt_gs[:,:2].count_hom_alt()

In [ ]:
cod_var_gs_r = cod_var_gs.reset_index()

# tlx
cod_var_gs_tlx = cod_var_gs_r[tlx_homalt]

# tap
cod_var_gs_tap = cod_var_gs_r[tap_homalt]

In [ ]:
cols = ['CHROM', 'POS', 'REF', 'ALT_1', 'ALT_2',
        'ANN_Annotation',
        'ANN_Annotation_Impact',
        'ANN_Feature_ID',
        'ANN_Feature_Type',
        'ANN_Gene_ID',
        'ANN_Gene_Name',
        'ANN_Rank',
        'ANN_Transcript_BioType']

cod_var_gs_tlx[cols]

In [ ]:
cod_var_gs_tap[cols]

# CASE 2: Variant in enhancers

In [ ]:
import pybedtools as pb

In [ ]:
enh = pb.BedTool('tracks/Enhancers_ChromHMM.bed')
enh_df = enh.to_dataframe()

In [ ]:
#enh_df.head()

In [ ]:
ftlx = 'tracks/WGS-WES/Germline/FERRIER_09_Germline.allchr.snpEff.p.SAL.SAL10_1.vcf'
var_b = pb.BedTool(ftlx)

In [ ]:
var_enh = (var_b + enh).saveas('tracks/WGS-WES/Germline/Vars_Enh_noHeader.vcf')

In [ ]:
len(var_enh)

In [ ]:
# Concat with header

# !cat tracks/WGS-WES/Germline/Germline_header.txt tracks/WGS-WES/Germline/Vars_Enh_noHeader.vcf > tracks/WGS-WES/Germline/Vars_Enh.vcf

In [ ]:
var_enh_df = var_enh.to_dataframe(names=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT','AC3812','AC3813','AC3814','AC3815'])

In [ ]:
var_enh_df.head()

In [ ]:
#var_enh_df['INFO'][16]

In [ ]:
#plot_variant_hist_2d('QD', 'MQ', var, downsample=500)



#“MQ” is average mapping quality across all samples.
#plot_variant_hist('MQ', var, down=2)

#“QD” is a slightly odd statistic but turns out to be very useful 
# for finding poor quality SNPs. Roughly speaking, high numbers 
# mean that evidence for variation is strong (concentrated), 
# low numbers mean that evidence is weak (dilute).


#x = var['QD'][:][::1000]
#plot_variant_hist('QD', var, bins=30, down=500)

#ac = gt.count_alleles()
#ac

